# Random Forest Hyper-parametrization

In [1]:
import numpy as np
import pandas as pd
import sys

from data_processor import sentence_to_words, extract_BoW_features
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/Life/jsaldiva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/Life/jsaldiva/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load data

In [2]:
data = pd.read_csv('./data/train_test_data.csv')

### Separate labels from data

In [3]:
X, y = data['sentence'], data['label']

## Prepare data

### Normalize sentences

In [4]:
%%time
X_clean = [sentence_to_words(sentence) for sentence in X]
X_clean_with_steeming = [sentence_to_words(sentence, steeming=True) for sentence in X]
X_clean_with_lemmatization = [sentence_to_words(sentence, lemmatization=True) for sentence in X]
X_clean_with_steeming_lemmatization = [sentence_to_words(sentence, True, True) for sentence in X]

CPU times: user 14.9 s, sys: 1.04 s, total: 16 s
Wall time: 16 s


In [5]:
preprocess_data = {
    'clean': {'data': X_clean, 'labels': y},
    'clean_steeming': {'data': X_clean_with_steeming, 'labels': y},
    'clean_lemmatization': {'data': X_clean_with_lemmatization, 'labels': y},
    'clean_steeming_lemmatization': {'data': X_clean_with_steeming_lemmatization, 'labels': y}
}

### Transform sentences

In [6]:
transformations = ['tc', 'tf', 'tfidf']
max_features=[100, 200, 400, 500, 800]
ngram_ranges=[(1,1),(1,2),(1,3)]
total_cycles = len(transformations)*len(max_features)*len(ngram_ranges)*len(preprocess_data)

In [7]:
transformed_txts = []

In [8]:
with tqdm(total=total_cycles, file=sys.stdout) as pbar:
    for transformation in transformations:        
        for max_feature in max_features:
            for ngram_range in ngram_ranges:
                for data_type, data_dict in preprocess_data.items():
                    pbar.update(1)
                    data = data_dict['data']
                    transformed_data, _ = extract_BoW_features(data, max_feature, transformation, ngram_range)
                    transformed_txts.append(
                        {
                            'transformation': transformation,
                            'max_features': max_feature,
                            'ngram_range': ngram_range,
                            'data': transformed_data,
                            'data_type': data_type,
                            'labels': data_dict['labels']
                        }
                    )

100%|██████████| 180/180 [00:13<00:00, 13.22it/s]


## Set parameters

In [9]:
n_estimators = [int(x) for x in np.linspace(start=100, stop=1000, num=10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 7, 10]
min_samples_leaf = [1, 2, 3]
bootstrap = [True, False]

### Set parameters grid 

In [10]:
param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}

## Run hyper-parametrization

In [12]:
outputs = []
with tqdm(total=len(transformed_txts), file=sys.stdout) as pbar:
    for transformed_txt in transformed_txts:
        pbar.update(1)
        transformed_X = transformed_txt['data']
        y = transformed_txt['labels']
        X_train, _, y_train, _ = train_test_split(transformed_X, y, test_size=0.20, random_state=42)
        rf_classifier = RandomForestClassifier()
        rf_random = RandomizedSearchCV(estimator = rf_classifier, param_distributions = param_grid, n_iter = 50, cv = 5, verbose=2, random_state=42, n_jobs = -1)
        rf_random.fit(X_train, y_train)
        loop_dict = {
            'transformation': transformed_txt['transformation'],
            'max_features_txt': transformed_txt['max_features'],
            'ngram_range': transformed_txt['ngram_range'],
            'data_type': transformed_txt['data_type']            
        }
        loop_dict.update(rf_random.best_params_)
        outputs.append(loop_dict)

  0%|          | 0/180 [00:00<?, ?it/s]Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   27.9s finished


  1%|          | 2/180 [00:29<43:18, 14.60s/it]Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   27.7s finished


  2%|▏         | 3/180 [00:57<55:18, 18.75s/it]Fitting 5 folds for each of 50 candidates, totalling 250 fits


...

 99%|█████████▉| 179/180 [1:58:53<00:51, 51.46s/it]Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   30.9s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   50.0s finished


100%|██████████| 180/180 [1:59:45<00:00, 51.72s/it]Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   48.1s finished


100%|██████████| 180/180 [2:00:35<00:00, 40.20s/it]


## Save hyperparameters

Let's save hypterparameters to be used during training

In [13]:
df_outputs = pd.DataFrame(outputs)

In [14]:
df_outputs.head()

,transformation,max_features_txt,ngram_range,data_type,n_estimators,min_samples_split,min_samples_leaf,max_features,max_depth,bootstrap
0,tc,100,"(1, 1)",clean,1000,10,1,auto,20.0,False
1,tc,100,"(1, 1)",clean_steeming,500,7,1,auto,20.0,False
2,tc,100,"(1, 1)",clean_lemmatization,700,7,2,sqrt,NaN,False
3,tc,100,"(1, 1)",clean_steeming_lemmatization,400,2,2,sqrt,60.0,True
4,tc,100,"(1, 2)",clean,700,7,2,sqrt,NaN,False


In [15]:
df_outputs.to_csv('./data/rf_parameters.csv', index=False)